# How It Works

   ## Video Processing:

   The video is opened using cv2.VideoCapture.

   Each frame is processed to detect the cat face and extract features.

   ## Feature Extraction:

   The extract_features function extracts HSV and LBP features from the detected cat face.

   ## Prediction:

   The trained model predicts the cat label for each frame.

   ## Majority Voting:

   The predictions for all frames are counted, and the majority label is determined using Counter.

   ## Output:

   The majority label is printed as the final classification for the video.

In [1]:
import cv2
import numpy as np
import joblib
from skimage.feature import local_binary_pattern
from collections import Counter

# Configuration Constants
HAAR_CASCADE_PATH = 'haarcascade_frontalcatface.xml'
GRID_SIZE = (4, 4)  # Split face into 4x4 grid
FACE_SIZE = (100, 100)  # Standard size for face resizing

# Load the trained model
model = joblib.load('cat_classifier.pkl')

# Load the Haar Cascade classifier
face_cascade = cv2.CascadeClassifier(HAAR_CASCADE_PATH)
if face_cascade.empty():
    raise FileNotFoundError("Could not load Haar Cascade file. Check the path.")


def extract_features(frame):
    """
    Extract distinguishing features from a frame containing a cat face
    Returns feature vector or None if no face detected
    """
    # Convert to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect cat faces using Haar Cascade
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30)
    )

    if len(faces) == 0:
        return None

    # Take largest face (assuming single cat per frame)
    (x, y, w, h) = max(faces, key=lambda f: f[2] * f[3])
    face_roi = frame[y:y + h, x:x + w]

    # Resize face to standard size for consistent feature extraction
    face_roi = cv2.resize(face_roi, FACE_SIZE)

    # --- Color Features (HSV Space) ---
    hsv = cv2.cvtColor(face_roi, cv2.COLOR_BGR2HSV)
    cell_height = FACE_SIZE[0] // GRID_SIZE[0]
    cell_width = FACE_SIZE[1] // GRID_SIZE[1]

    hsv_features = []
    for i in range(GRID_SIZE[0]):
        for j in range(GRID_SIZE[1]):
            # Extract grid cell
            y_start = i * cell_height
            y_end = (i + 1) * cell_height
            x_start = j * cell_width
            x_end = (j + 1) * cell_width

            cell = hsv[y_start:y_end, x_start:x_end]
            # Calculate mean HSV values for the cell
            hsv_features.extend(np.mean(cell, axis=(0, 1)))

    # --- Texture Features (LBP) ---
    gray_face = cv2.cvtColor(face_roi, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(
        gray_face,
        P=8,  # 8 surrounding points
        R=1,  # Radius of 1 pixel
        method='uniform'
    )
    # Create histogram of LBP patterns (16 bins)
    lbp_hist, _ = np.histogram(
        lbp,
        bins=16,
        range=(0, 16)
    )

    # Combine all features into single array
    return np.concatenate([hsv_features, lbp_hist])


# Open the video file
video_path = "demofootage.mov"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    raise FileNotFoundError(f"Could not open video file: {video_path}")

# Initialize a list to store predictions
predictions = []

# Process each frame of the video
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Extract features from the frame
    features = extract_features(frame)

    if features is not None:
        # Predict the cat label using the trained model
        prediction = model.predict([features])[0]
        predictions.append(prediction)
        frame_count += 1

    # Optional: Display the frame with the predicted label (for debugging)
    if features is not None:
        cv2.putText(frame, f"Predicted: {prediction}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit early
        break

cap.release()
cv2.destroyAllWindows()

# Determine the majority prediction
if predictions:
    majority_label = Counter(predictions).most_common(1)[0][0]
    print(f"This is a video clip of {majority_label}.")
else:
    print("No cat faces detected in the video.")

KeyboardInterrupt: 

### Press 'q' to quit early

# Local Binary Patterns (LBP)
LBP captures the local texture information by comparing each pixel in an image with its neighboring pixels. It generates a binary code for each pixel based on whether its neighbors are brighter or darker than itself.

## Steps to Compute LBP
### Step 1: Define a Neighborhood

For each pixel in the image, consider a small neighborhood around it (e.g., a 3x3 grid).

The center pixel is the one being analyzed, and the 8 surrounding pixels are its neighbors.

### Step 2: Compare Neighbors to the Center Pixel

Compare the intensity value of each neighbor to the intensity value of the center pixel.

If the neighbor's intensity is greater than or equal to the center pixel's intensity, assign a 1 to that neighbor.

If the neighbor's intensity is less than the center pixel's intensity, assign a 0.

### Step 3: Generate the Binary Pattern

 Starting from a fixed position (e.g., top-left neighbor), collect the binary values (0s and 1s) from all neighbors in a clockwise or counterclockwise order.

 Convert this binary sequence into a decimal number. This decimal number is the LBP code for the center pixel.

### Step 4: Repeat for All Pixels

Repeat the above steps for every pixel in the image to generate an LBP-encoded image.



In [ ]:
import cv2
import numpy as np
import joblib
from skimage.feature import local_binary_pattern
from collections import Counter

# Configuration Constants
HAAR_CASCADE_PATH = 'haarcascade_frontalcatface.xml'
GRID_SIZE = (4, 4)  # Split face into 4x4 grid
FACE_SIZE = (100, 100)  # Standard size for face resizing

# Load the trained model
model = joblib.load('cat_classifier.pkl')

# Load the Haar Cascade classifier
face_cascade = cv2.CascadeClassifier(HAAR_CASCADE_PATH)
if face_cascade.empty():
    raise FileNotFoundError("Could not load Haar Cascade file. Check the path.")


def extract_features(frame):
    """
    Extract distinguishing features from a frame containing a cat face
    Returns feature vector or None if no face detected
    """
    # Convert to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect cat faces using Haar Cascade
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30)
    )

    if len(faces) == 0:
        return None

    # Take largest face (assuming single cat per frame)
    (x, y, w, h) = max(faces, key=lambda f: f[2] * f[3])
    face_roi = frame[y:y + h, x:x + w]

    # Resize face to standard size for consistent feature extraction
    face_roi = cv2.resize(face_roi, FACE_SIZE)

    # --- Color Features (HSV Space) ---
    hsv = cv2.cvtColor(face_roi, cv2.COLOR_BGR2HSV)
    cell_height = FACE_SIZE[0] // GRID_SIZE[0]
    cell_width = FACE_SIZE[1] // GRID_SIZE[1]

    hsv_features = []
    for i in range(GRID_SIZE[0]):
        for j in range(GRID_SIZE[1]):
            # Extract grid cell
            y_start = i * cell_height
            y_end = (i + 1) * cell_height
            x_start = j * cell_width
            x_end = (j + 1) * cell_width

            cell = hsv[y_start:y_end, x_start:x_end]
            # Calculate mean HSV values for the cell
            hsv_features.extend(np.mean(cell, axis=(0, 1)))

    # --- Texture Features (LBP) ---
    gray_face = cv2.cvtColor(face_roi, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(
        gray_face,
        P=8,  # 8 surrounding points
        R=1,  # Radius of 1 pixel
        method='uniform'
    )
    # Create histogram of LBP patterns (16 bins)
    lbp_hist, _ = np.histogram(
        lbp,
        bins=16,
        range=(0, 16)
    )

    # Combine all features into single array
    return np.concatenate([hsv_features, lbp_hist])


# Open the video file
video_path = "demofootage2.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    raise FileNotFoundError(f"Could not open video file: {video_path}")

# Initialize a list to store predictions
predictions = []

# Process each frame of the video
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Extract features from the frame
    features = extract_features(frame)

    if features is not None:
        # Predict the cat label using the trained model
        prediction = model.predict([features])[0]
        predictions.append(prediction)
        frame_count += 1

    # Optional: Display the frame with the predicted label (for debugging)
    if features is not None:
        cv2.putText(frame, f"Predicted: {prediction}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit early
        break

cap.release()
cv2.destroyAllWindows()

# Determine the majority prediction
if predictions:
    majority_label = Counter(predictions).most_common(1)[0][0]
    print(f"This is a video clip of {majority_label}.")
else:
    print("No cat faces detected in the video.")

# Hue, Saturation, Value (HSV)
is a color space that separates color information into three distinct components, making it more intuitive and useful for certain image processing tasks compared to the RGB color space. Here's a detailed explanation of how HSV features work:
## 1. What is HSV?

HSV is an alternative representation of the RGB color space. It separates color information into:

Hue (H): Represents the dominant wavelength of the color (e.g., red, green, blue).

Saturation (S): Represents the purity or intensity of the color (e.g., pastel vs. vivid).

Value (V): Represents the brightness of the color (e.g., dark vs. bright).

## 2. Why Use HSV?

Intuitive: Separates color (Hue) from brightness (Value) and color intensity (Saturation).

Robustness: Less sensitive to lighting variations compared to RGB.

Useful for Segmentation: Easier to isolate specific colors or ranges of colors.

In [ ]:
import cv2
import numpy as np
import joblib
from skimage.feature import local_binary_pattern
from collections import Counter

# Configuration Constants
HAAR_CASCADE_PATH = 'haarcascade_frontalcatface.xml'
GRID_SIZE = (4, 4)  # Split face into 4x4 grid
FACE_SIZE = (100, 100)  # Standard size for face resizing

# Load the trained model
model = joblib.load('cat_classifier.pkl')

# Load the Haar Cascade classifier
face_cascade = cv2.CascadeClassifier(HAAR_CASCADE_PATH)
if face_cascade.empty():
    raise FileNotFoundError("Could not load Haar Cascade file. Check the path.")


def extract_features(frame):
    """
    Extract distinguishing features from a frame containing a cat face
    Returns feature vector or None if no face detected
    """
    # Convert to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect cat faces using Haar Cascade
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30)
    )

    if len(faces) == 0:
        return None

    # Take largest face (assuming single cat per frame)
    (x, y, w, h) = max(faces, key=lambda f: f[2] * f[3])
    face_roi = frame[y:y + h, x:x + w]

    # Resize face to standard size for consistent feature extraction
    face_roi = cv2.resize(face_roi, FACE_SIZE)

    # --- Color Features (HSV Space) ---
    hsv = cv2.cvtColor(face_roi, cv2.COLOR_BGR2HSV)
    cell_height = FACE_SIZE[0] // GRID_SIZE[0]
    cell_width = FACE_SIZE[1] // GRID_SIZE[1]

    hsv_features = []
    for i in range(GRID_SIZE[0]):
        for j in range(GRID_SIZE[1]):
            # Extract grid cell
            y_start = i * cell_height
            y_end = (i + 1) * cell_height
            x_start = j * cell_width
            x_end = (j + 1) * cell_width

            cell = hsv[y_start:y_end, x_start:x_end]
            # Calculate mean HSV values for the cell
            hsv_features.extend(np.mean(cell, axis=(0, 1)))

    # --- Texture Features (LBP) ---
    gray_face = cv2.cvtColor(face_roi, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(
        gray_face,
        P=8,  # 8 surrounding points
        R=1,  # Radius of 1 pixel
        method='uniform'
    )
    # Create histogram of LBP patterns (16 bins)
    lbp_hist, _ = np.histogram(
        lbp,
        bins=16,
        range=(0, 16)
    )

    # Combine all features into single array
    return np.concatenate([hsv_features, lbp_hist])


# Open the video file
video_path = "demofootage3.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    raise FileNotFoundError(f"Could not open video file: {video_path}")

# Initialize a list to store predictions
predictions = []

# Process each frame of the video
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Extract features from the frame
    features = extract_features(frame)

    if features is not None:
        # Predict the cat label using the trained model
        prediction = model.predict([features])[0]
        predictions.append(prediction)
        frame_count += 1

    # Optional: Display the frame with the predicted label (for debugging)
    if features is not None:
        cv2.putText(frame, f"Predicted: {prediction}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit early
        break

cap.release()
cv2.destroyAllWindows()

# Determine the majority prediction
if predictions:
    majority_label = Counter(predictions).most_common(1)[0][0]
    print(f"This is a video clip of {majority_label}.")
else:
    print("No cat faces detected in the video.")